# Imports

In [ ]:
import time
import os
import copy

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import accuracy_score

%matplotlib inline  
import matplotlib.pyplot as plt

# Gather data

In [ ]:
# Download image data of bees and ants.
! wget https://download.pytorch.org/tutorial/hymenoptera_data.zip

In [ ]:
# unzip the data.
! unzip hymenoptera_data.zip

# Pre-process the data

In [ ]:
PATH = 'hymenoptera_data'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'val')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

In [ ]:
# Take a look at some of our images and the corresponding labels.
class_names = train_dataset.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# Create a test set in addition to the validation set
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

# Load the pre-trained model and create a classification head

In [ ]:
# Load the pre-trained MobileNet V2 without the classification head
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
# "freeze" the convolutional, feature extraction parameters.
base_model.trainable = False

In [ ]:
# Create a data augmentation layer because of our small data set size.
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Create a rescaling layer because the MobileNet model expects pixel vaues in a  
# range of [-1,1], but our values are in teh rannge [0,255].
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

# Create a classification head for our model
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

In [ ]:
# Put everything together using the keras functional API
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = rescale(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# Prep the model for training and set training parameters.
learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model (using transfer learning)

In [ ]:
epochs = 50
history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=validation_dataset)

In [ ]:
# Plot the training loss for each epoch to see how the model converged.
history_dict = history.history
plt.plot(list(range(0,epochs)), history_dict['loss'], 'bo--', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Evaluate the transfer learned model

In [ ]:
# Evaluate the model on the test set. 
test_loss, test_acc = model.evaluate(test_dataset, 
          verbose=2)

print('\nTest accuracy:', test_acc)

# Update the training to "fine-tune" the base model

In [ ]:
# Reset the base model to be "trainable"
base_model.trainable = True

In [ ]:
# Print the number of layers in the base model
len(base_model.layers)

In [ ]:
# Let's keep 100 of these layers frozen, and fine-tune the rest.
for layer in base_model.layers[:100]:
  layer.trainable =  False

In [ ]:
# Prep the model for re-training. This time with a smaller learning rate.
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate/10),
              metrics=['accuracy'])

# Fine tune the model 

In [ ]:
ft_epochs =  epochs + 50
history_ft = model.fit(train_dataset,
                         epochs=ft_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

In [ ]:
# Plot the training loss for each epoch to see how the model converged.
history_dict = history.history
plt.plot(list(range(0,ft_epochs - epochs)), history_dict['loss'], 'bo--', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Evaluate the fine-tuned model

In [ ]:
# Evaluate the model on the test set. 
test_loss, test_acc = model.evaluate(test_dataset, 
          verbose=2)

print('\nTest accuracy:', test_acc)